<a href="https://colab.research.google.com/github/swathimurikipudi/analyzing-the-performance-and-efficiency-of-radisson-hotels-using-data-visualization-techniques/blob/main/radissonhotels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import csv

# Data Preprocessing
import re
import string
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import nltk as nlp
import nltk.corpus
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.util import everygrams
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import pickle as pk
from scipy import sparse as sp

# Text Manipulation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

# For general visualizations / text analysis
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from wordcloud import WordCloud
import plotly.express as px
import plotly.figure_factory as ff
plt.style.use('fivethirtyeight')

# For classification and sentiment analysis
from textblob import TextBlob
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score, precision_score, confusion_matrix, classification_report, roc_auc_score, roc_curve,auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
df = pd.read_excel('/content/sample_data/Capstone2_FinalReport_Dataset_Clean.xlsx')
df.head(20)

,Unnamed: 0,Country,Date,Rating,Travel Status,Review,YDate,MDate,Review_clean,Subjectivity,Polarity,Sentiment,Analysis,SentimentML
0,0,Singapore,2020-02-28,5,Solo,"Great location, comfortable and clean room, qu...",2020,2,great location comfortable clean room quiet ye...,0.580556,0.394444,Positive,1,Positive
1,1,Singapore,2020-02-27,5,Business,Stayed at the Sunway Pyramid Hotel for 3 night...,2020,2,stayed sunway pyramid hotel night february roo...,0.377273,0.568182,Positive,1,Positive
2,2,Mauritius,2020-02-27,4,Business,"Great for couples, families or just a business...",2020,2,great couple family business trip area,0.750000,0.800000,Positive,1,Positive
3,3,Singapore,2020-02-25,5,Friends,The rooms are newly renovated with great furni...,2020,2,room newly renovated great furnishing workmans...,0.438026,0.228208,Positive,1,Positive
4,4,Brunei,2020-02-25,5,Friends,Everything you need is within close proximity ...,2020,2,everything need within close proximity area co...,0.500000,0.375000,Positive,1,Positive
5,5,Brunei,2020-02-24,5,Friends,Very convenient. You don't have to go out of t...,2020,2,convenient go hotel connected mall activity fo...,0.712500,0.304167,Positive,1,Positive
6,6,Malaysia,2020-02-24,4,Business,"Helpful staff, great hospitality n room was we...",2020,2,helpful staff great hospitality room well main...,0.548333,0.426667,Positive,1,Positive
7,7,Malaysia,2020-02-23,1,Solo,New building with poor service and unfriendly ...,2020,2,new building poor service unfriendly manager,0.527273,-0.131818,Negative,-1,Negative
8,8,Malaysia,2020-02-23,4,Solo,No late checkout option even if you checkin at...,2020,2,late checkout option even checkin,0.600000,-0.300000,Negative,-1,Negative
9,9,Mauritius,2020-02-23,4,Solo,Now friendly staff,2020,2,friendly staff,0.500000,0.375000,Positive,1,Positive


In [4]:
# prints the number of variables and observations of the listings dataset.
print('Number of variables: {}'.format(df.shape[1]))
print('Number of observations: {}'.format(df.shape[0]))

Number of variables: 14
Number of observations: 1975


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     1975 non-null   int64         
 1   Country        1975 non-null   object        
 2   Date           1975 non-null   datetime64[ns]
 3   Rating         1975 non-null   int64         
 4   Travel Status  1975 non-null   object        
 5   Review         1975 non-null   object        
 6   YDate          1975 non-null   int64         
 7   MDate          1975 non-null   int64         
 8   Review_clean   1974 non-null   object        
 9   Subjectivity   1975 non-null   float64       
 10  Polarity       1975 non-null   float64       
 11  Sentiment      1975 non-null   object        
 12  Analysis       1975 non-null   int64         
 13  SentimentML    1975 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(6)
memory usage: 216.

In [6]:
df.isnull().sum()

Unnamed: 0       0
Country          0
Date             0
Rating           0
Travel Status    0
Review           0
YDate            0
MDate            0
Review_clean     1
Subjectivity     0
Polarity         0
Sentiment        0
Analysis         0
SentimentML      0
dtype: int64

DATA PREPROCESSING

In [9]:
# Change the datatype of Date and Feature it into Year and Month
import datetime

df['Date'] = pd.to_datetime(df['Date'])

df['YDate'], df['MDate'] = df['Date'].dt.year, df['Date'].dt.month

df

,Unnamed: 0,Country,Date,Rating,Travel Status,Review,YDate,MDate,Review_clean,Subjectivity,Polarity,Sentiment,Analysis,SentimentML
0,0,Singapore,2020-02-28,5,Solo,"Great location, comfortable and clean room, qu...",2020,2,great location comfortable clean room quiet ye...,0.580556,0.394444,Positive,1,Positive
1,1,Singapore,2020-02-27,5,Business,Stayed at the Sunway Pyramid Hotel for 3 night...,2020,2,stayed sunway pyramid hotel night february roo...,0.377273,0.568182,Positive,1,Positive
2,2,Mauritius,2020-02-27,4,Business,"Great for couples, families or just a business...",2020,2,great couple family business trip area,0.750000,0.800000,Positive,1,Positive
3,3,Singapore,2020-02-25,5,Friends,The rooms are newly renovated with great furni...,2020,2,room newly renovated great furnishing workmans...,0.438026,0.228208,Positive,1,Positive
4,4,Brunei,2020-02-25,5,Friends,Everything you need is within close proximity ...,2020,2,everything need within close proximity area co...,0.500000,0.375000,Positive,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,1970,Philippines,2019-03-01,5,Business,Front Desk needs a little bit more warmthÂ,2019,3,front desk need little bit warmth,0.500000,-0.187500,Negative,-1,Negative
1971,1971,Philippines,2019-03-01,5,Business,Airport shuttle services shows 90MYR but was a...,2019,3,airport shuttle service show myr actually quot...,0.340000,0.240000,Positive,1,Positive
1972,1972,Philippines,2019-03-01,5,Business,Gym is a bit small but has enough equipment to...,2019,3,gym bit small enough equipment get workout war...,0.550000,0.025000,Positive,1,Positive
1973,1973,Philippines,2019-03-01,5,Business,Hotel was still on soft opening and swimming p...,2019,3,hotel still soft opening swimming pool still c...,0.350000,0.100000,Positive,1,Positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')